In [ ]:
from skt.gcp import (
    PROJECT_ID,
    bq_insert_overwrite,
    bq_to_df,
    bq_to_pandas,
    get_bigquery_client,
    bq_table_exists,
    get_max_part,
    load_query_result_to_table,
    pandas_to_bq,
    pandas_to_bq_table,
    load_bigquery_ipython_magic,
    get_bigquery_client,
    _print_query_job_results,
    load_query_result_to_partitions
    
)

from skt.ye import (
    get_hdfs_conn,
    get_spark,
    hive_execute,
    hive_to_pandas,
    pandas_to_parquet,
    slack_send,
    get_secrets
)

In [ ]:
from google.cloud.bigquery.job import QueryJobConfig

In [ ]:
import pandas as pd
import requests
import json
from datetime import datetime, date, timedelta
from typing import List, Dict

In [ ]:
db_name = 'adot_reco_dev'
project_id = 'skt-datahub'

In [ ]:
bq_client = get_bigquery_client()

In [ ]:
url = "http://insight.nugu.co.kr/psionic/v1/api/export/edf_1718241386286"
headers = {
    "Content-Type": "application/json", 
    "token": nudge_token
}

In [ ]:
response = requests.post(url, headers=headers, data="{}")
if int(response.status_code) != 200:
    raise RuntimeError(f"api request failed: {response.status_code}")


In [ ]:
api_text = json.loads(response.text)

In [ ]:
def get_api_cols(infos:List[Dict]):
    result = list()
    for info in infos:
        result.append(info['name'])
    return result

In [ ]:
api_info = api_text['info']
api_data = api_text['data']
col_names = get_api_cols(api_info)

In [ ]:
api_final_result = []
for api_datum in api_data:
    temp_dict = dict()
    data = api_datum['row']
    for col_name, datum in zip(col_names, data):
        if col_name == 'event_id_description' and (datum is None or datum ==""):
            temp_dict = None
            break
        elif col_name =='variable':
            datum_str = json.loads(datum)
            datum_text = datum_str['titleTextPolite'].replace("<br>", " ")
            temp_dict[col_name] = datum_text
        else:
            temp_dict[col_name] = datum
    if temp_dict is not None:
        temp_dict['dt'] = current_dt_next
        api_final_result.append(temp_dict)

In [ ]:
api_data = pd.DataFrame(api_final_result)

In [ ]:
table_nm = 'nudge_offering_api_table'

In [ ]:
table_exists = bq_table_exists(table=f'{db_name}.{table_nm}', project_id=project_id)

In [ ]:
def create_table_query(db_name:str, table_nm:str, col_names:List, PROJECT_ID:str=PROJECT_ID) -> str:
    query = f"CREATE TABLE IF NOT EXISTS {PROJECT_ID}.{db_name}.{table_nm}(\n"
    
    for col_name in col_names:
        if col_name == "dt":  # 'dt' 열은 Date 타입
            query += f"            {col_name} DATE,\n"
        else:  # 그 외 열은 String 타입
            query += f"            {col_name} STRING,\n"
    
    query = query.rstrip(',\n') + '\n)'    
    return query

In [ ]:
if not table_exists:
    query = create_table_query(db_name=db_name, table_nm=table_nm, col_names=col_names)
    query_job = bq_client.query(query)
    query_job.result()
    print(f"생성된 테이블 : {db_name}.{table_nm}")
else:
    """ 기존 테이블의 스키마와 비교 후 안맞으면 에러"""
    table_ref = bq_client.dataset(db_name).table(table_nm)
    table = bq_client.get_table(table_ref)

    # Get the schema
    schema = table.schema

    #Extract column names
    column_names = [field.name for field in schema]

    new_columns = []
    for col_nm in col_names:
        if col_nm not in column_names:
            new_columns.append(col_nm)
    
    if new_columns:
        raise TypeError(f"new column:{new_columns} added from api")

In [ ]:
pandas_to_bq(pd_df=api_data, destination=f'{project_id}.{db_name}.{table_nm}', overwrite=True)